In [14]:
sealed class MyList[A]()
case class MyNil[A]() extends MyList[A]
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A]

def length(x: MyList[Int]): Int =
    x match {
        case MyNil() => 0
        case MyCons(hd, tl) => 1 + length(tl)
    }

val t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
length(t)

defined class MyList
defined class MyNil
defined class MyCons
defined function length
t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
res13_5: Int = 2

In [2]:
// cannot use
class MyList[A]()
class MyNil[A]() extends MyList[A]
class MyCons[A](val hd: A, val tl: MyList[A]) extends MyList[A]

val t: MyList[Int] = new MyCons(3, new MyCons(4, new MyNil()))

def length[A](l: MyList[A]) = {
/// ???    
}

defined class MyList
defined class MyNil
defined class MyCons
t: MyList[Int] = ammonite.$sess.cmd1$Helper$MyCons@58a07d06
defined function length

### To implement `length` method (Elimination)

In [6]:
class MyList[A]() {
    // need to be removed
    def getHead(): Option[A] = None
    def getTail(): MyList[A] = new MyNil()
}
class MyNil[A]() extends MyList[A] {
    override def getHead(): Option[A] = None
    override def getTail(): MyList[A] = new MyNil()
    
}
class MyCons[A](val hd: A, val tl: MyList[A]) extends MyList[A] {
    override def getHead(): Option[A] = Some(hd)
    override def getTail(): MyList[A] = tl
}

val t: MyList[Int] = new MyCons(3, new MyCons(4, new MyNil()))

def length[A](l: MyList[A]): Int = {
    l.getHead() match {
        case None => 0
        case Some(hd) => 1+ length(l.getTail())
    }
}

length(t)
length(new MyList())

defined class MyList
defined class MyNil
defined class MyCons
t: MyList[Int] = ammonite.$sess.cmd5$Helper$MyCons@3fa1087e
defined function length
res5_5: Int = 2
res5_6: Int = 0

### Abstract Class
* can implement method only using interface

In [11]:
abstract class MyList[A]() {
    def getHead(): Option[A]
    def getTail(): MyList[A]
}

def length[A](l: MyList[A]): Int = {
    l.getHead() match {
        case None => 0
        case Some(hd) => 1+ length(l.getTail())
    }
}

defined class MyList
defined function length

In [12]:
class MyNil[A]() extends MyList[A] {
    override def getHead(): Option[A] = None
//     override def getTail(): MyList[A] = new MyNil()
    override def getTail(): MyNil[A] = new MyNil()
    
}
class MyCons[A](val hd: A, val tl: MyList[A]) extends MyList[A] {
    override def getHead(): Option[A] = Some(hd)
    override def getTail(): MyList[A] = tl
}

defined class MyNil
defined class MyCons

### Example Interface

In [20]:
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}

defined class Iter

In [21]:
def sumElements[A](f: A=>Int)(xs: Iter[A]) : Int =
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }

def sumElementsId(xs:Iter[Int]) =
    sumElements((x:Int)=>x)(xs)

defined function sumElements
defined function sumElementsId

#### TO USE
> create wrapper or implement orignial class

In [21]:
//written by Bob
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
def sumElements[A](f: A=>Int)(xs: Iter[A]) : Int =
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }

def sumElementsId(xs:Iter[Int]) =
    sumElements((x:Int)=>x)(xs)

//written by Alice
sealed abstract class MyList[A]()
case class MyNil[A]() extends MyList[A]
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A]

val t1 = MyCons(3, MyCons(5, MyCons(7, MyNil())))
sumElementsId(t1)

cmd21.sc:20: type mismatch;
 found   : Helper.this.MyCons[Int]
 required: Helper.this.Iter[Int]
val res21_7 = sumElementsId(t1)
                            ^Compilation Failed

: 

In [23]:

//written by Bob
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
def sumElements[A](f: A=>Int)(xs: Iter[A]) : Int =
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }

def sumElementsId(xs:Iter[Int]) =
    sumElements((x:Int)=>x)(xs)

//written by Alice
sealed abstract class MyList[A] extends Iter[A]()
case class MyNil[A]() extends MyList[A] {
    def getValue: Option[A] = None
    def getNext: Iter[A] = this
}
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A] {
    def getValue: Option[A] = Some(hd)
    def getNext: Iter[A] = tl
}
val t1 = MyCons(3, MyCons(5, MyCons(7, MyNil())))
sumElementsId(t1)
sumElements((x: Int)=>x*x)(t1)


defined class Iter
defined function sumElements
defined function sumElementsId
defined class MyList
defined class MyNil
defined class MyCons
t1: MyCons[Int] = MyCons(3, MyCons(5, MyCons(7, MyNil())))
res22_7: Int = 15
res22_8: Int = 83

### Example: Count Down

In [26]:
// if getValue(i) return None, you should not use i.getNext()
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
def sumElements[A](f: A=>Int)(xs: Iter[A]) : Int =
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }

class IntCounter(n: Int) extends Iter[Int] {
    def getValue = if (n < 0) None else Some(n)
    def getNext = new IntCounter(n-1)
}

sumElements[Int]((x)=>x)(new IntCounter(100))

defined class Iter
defined function sumElements
defined class IntCounter
res25_3: Int = 5050

### Iter for MyTree

In [35]:
abstract class Iter[A] {
    def getValue: Option[A]
    def getNext: Iter[A]
}
def sumElements[A](f: A=>Int)(xs: Iter[A]) : Int =
    xs.getValue match {
        case None => 0
        case Some(n) => f(n) + sumElements(f)(xs.getNext)
    }


sealed abstract class MyTree[A] extends Iter[A]
    case class Empty[A]() extends MyTree[A] {
        def getValue = None
        def getNext = this
    }
    case class Node[A](value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A] {
        def getValue = Some(value)
        // hard to change visit order to in, post (now in pre)
        def getNext: MyTree[A] = {
            // merge right tree to rightmost node from left tree
            def merge(l: MyTree[A]): MyTree[A] = {
                l match {
                    case Empty() => right
                    case Node(v, lt, rt) => Node(v, lt, merge(rt))
                }
            }
            merge(left)
        }
    }

val t1 = Node(3, Node(7, Node(2, Empty(), Empty()), Empty()), Node(8, Empty(), Empty()))
sumElements[Int]((x)=>x)(t1)
sumElements[Int]((x)=>x*x)(t1)

defined class Iter
defined function sumElements
defined class MyTree
defined class Empty
defined class Node
t1: Node[Int] = Node(
  3,
  Node(7, Node(2, Empty(), Empty()), Empty()),
  Node(8, Empty(), Empty())
)
res34_6: Int = 20
res34_7: Int = 126